In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Practice Lab: Linear Regression\n",
    "\n",
    "Welcome to your first practice lab! In this lab, you will implement linear regression with one variable to predict profits for a restaurant franchise.\n",
    "\n",
    "\n",
    "# Outline\n",
    "- [ 1 - Packages ](#1)\n",
    "- [ 2 - Linear regression with one variable ](#2)\n",
    "  - [ 2.1 Problem Statement](#2.1)\n",
    "  - [ 2.2  Dataset](#2.2)\n",
    "  - [ 2.3 Refresher on linear regression](#2.3)\n",
    "  - [ 2.4  Compute Cost](#2.4)\n",
    "    - [ Exercise 1](#ex01)\n",
    "  - [ 2.5 Gradient descent ](#2.5)\n",
    "    - [ Exercise 2](#ex02)\n",
    "  - [ 2.6 Learning parameters using batch gradient descent ](#2.6)\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<a name=\"1\"></a>\n",
    "## 1 - Packages \n",
    "\n",
    "First, let's run the cell below to import all the packages that you will need during this assignment.\n",
    "- [numpy](www.numpy.org) is the fundamental package for working with matrices in Python.\n",
    "- [matplotlib](http://matplotlib.org) is a famous library to plot graphs in Python.\n",
    "- ``utils.py`` contains helper functions for this assignment. You do not need to modify code in this file.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from utils import *\n",
    "import copy\n",
    "import math\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2 -  Problem Statement\n",
    "\n",
    "Suppose you are the CEO of a restaurant franchise and are considering different cities for opening a new outlet.\n",
    "- You would like to expand your business to cities that may give your restaurant higher profits.\n",
    "- The chain already has restaurants in various cities and you have data for profits and populations from the cities.\n",
    "- You also have data on cities that are candidates for a new restaurant. \n",
    "    - For these cities, you have the city population.\n",
    "    \n",
    "Can you use the data to help you identify which cities may potentially give your business higher profits?\n",
    "\n",
    "## 3 - Dataset\n",
    "\n",
    "You will start by loading the dataset for this task. \n",
    "- The `load_data()` function shown below loads the data into variables `x_train` and `y_train`\n",
    "  - `x_train` is the population of a city\n",
    "  - `y_train` is the profit of a restaurant in that city. A negative value for profit indicates a loss.   \n",
    "  - Both `X_train` and `y_train` are numpy arrays."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "# load the dataset\n",
    "x_train, y_train = load_data()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### View the variables\n",
    "Before starting on any task, it is useful to get more familiar with your dataset.  \n",
    "- A good place to start is to just print out each variable and see what it contains.\n",
    "\n",
    "The code below prints the variable `x_train` and the type of the variable."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Type of x_train: <class 'numpy.ndarray'>\n",
      "First five elements of x_train are:\n",
      " [6.1101 5.5277 8.5186 7.0032 5.8598]\n"
     ]
    }
   ],
   "source": [
    "# print x_train\n",
    "print(\"Type of x_train:\",type(x_train))\n",
    "print(\"First five elements of x_train are:\\n\", x_train[:5]) "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "`x_train` is a numpy array that contains decimal values that are all greater than zero.\n",
    "- These values represent the city population times 10,000\n",
    "- For example, 6.1101 means that the population for that city is 61,101\n",
    "  \n",
    "Now, let's print `y_train`"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Type of y_train: <class 'numpy.ndarray'>\n",
      "First five elements of y_train are:\n",
      " [17.592   9.1302 13.662  11.854   6.8233]\n"
     ]
    }
   ],
   "source": [
    "# print y_train\n",
    "print(\"Type of y_train:\",type(y_train))\n",
    "print(\"First five elements of y_train are:\\n\", y_train[:5])  "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Similarly, `y_train` is a numpy array that has decimal values, some negative, some positive.\n",
    "- These represent your restaurant's average monthly profits in each city, in units of \\$10,000.\n",
    "  - For example, 17.592 represents \\$175,920 in average monthly profits for that city.\n",
    "  - -2.6807 represents -\\$26,807 in average monthly loss for that city."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Check the dimensions of your variables\n",
    "\n",
    "Another useful way to get familiar with your data is to view its dimensions.\n",
    "\n",
    "Please print the shape of `x_train` and `y_train` and see how many training examples you have in your dataset."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "The shape of x_train is: (97,)\n",
      "The shape of y_train is:  (97,)\n",
      "Number of training examples (m): 97\n"
     ]
    }
   ],
   "source": [
    "print ('The shape of x_train is:', x_train.shape)\n",
    "print ('The shape of y_train is: ', y_train.shape)\n",
    "print ('Number of training examples (m):', len(x_train))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The city population array has 97 data points, and the monthly average profits also has 97 data points. These are NumPy 1D arrays."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Visualize your data\n",
    "\n",
    "It is often useful to understand the data by visualizing it. \n",
    "- For this dataset, you can use a scatter plot to visualize the data, since it has only two properties to plot (profit and population). \n",
    "- Many other problems that you will encounter in real life have more than two properties (for example, population, average household income, monthly profits, monthly sales).When you have more than two properties, you can still use a scatter plot to see the relationship between each pair of properties.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAYAAAAEWCAYAAABv+EDhAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAAsTAAALEwEAmpwYAAAvNElEQVR4nO2debxeVXX3vz8QVBQIkMgMUcG2aOEKaQLUKkpfRWoBW8WpClql1pKbGHwr1Uou+rZFrQGiLf2goGARoXWAIg7I5FCIJDGE0RItyDwHUFFM7nr/2Odwzz155vvMz+/7+ZzP85x9hr3Ofc7da++11l5bEYExxpjRY7NeC2CMMaY3WAEYY8yIYgVgjDEjihWAMcaMKFYAxhgzolgBGGPMiGIFYJpG0h9Kuk3SLyQdJembko7ptVz9hKSQtFeL175N0nfaLVMvkXSTpEN6LYeZjjwPYDSQdDuwI7AR+CXwTeD4iPhFC/e6HLg4Ik6vcOxY4N0R8bIZCdwmskbnCuBXQAD3AKdExOc7XG8Ae0fEujrnzQX+F9giIjZ0UqZ+QdIEsFdE/EWvZRl1PAIYLf40Ip4L7A/MA/6+fIKkZzRwnz2Bm9osWye5J3vubYAPAp+VtE+PZRpoGnxPTJ9jBTCCRMTdpBHAS+Bpc8XfSLoNuC0re4+kdZIekXSxpF2y8p8CLwD+KzMBPVPSVZLeLen3gH8DDsqOrc+uOVzSzZKekHS3pA+UZcrus17SSwplcyQ9Kel5kmZLuiQ75xFJ35fU1Psbia8DjwL7ZHWeJumebDtN0jOzug+RdJekD0l6SNLtkt5WkO0qSe8u7B8r6QeV6pX0J5J+LOlxSXdmPeCc72Wf67O/2UHle0k6WNJ1kh7LPg8uyfExST/M/r7fkTS7ihz1numZkv5Z0s8l3S/p3yQ9u3TtByXdB1QcQWXvzS2ZLDdL2j8rv13SH0s6DPgQ8Kbsea+X9EZJq0r3WSLpokp1mPZhBTCCSNodOBz4caH4KGABqWF8FfBPwNHAzsAdwJcBIuKFwM/JRhMR8Zv8BhFxC/Be4Jrs2Kzs0FnAX0XE1iSlc0VZpuw+XwXeUig+Grg6Ih4ATgDuAuaQTFkfIpl0mnnuzSS9HpgF3AB8GDgQGAP2A+YzfVS0EzAb2BU4BjhT0u80U2fGL4F3ZPX+CfDXko7Kjr08+5yV/c2uKcm8PfANYDmwA7AM+IakHQqnvRV4J/A8YEtgEwXb4DOdAryI9PfYKzvnpNK125NGgMeVbyzpjcBE9qzbAEcADxfPiYhvAf8IXJA9737AxcDzsw5EztuBc2s8h2kDVgCjxdezXvkPgKtJ/4g5/xQRj0TEk8DbgLMjYnXWMP8dqVc/t8V6f0tSLNtExKMRsbrKeV8C3lzYf2tWlt9jZ2DPiPhtRHw/Gndg7ZI990PAUuDtEfET0nN+NCIeiIgHgZNJDU+Rj0TEbyLialJDfHSDdT5NRFwVETdExGRErAXOB17R4OV/AtwWEV+MiA0RcT5wK/CnhXM+HxH/k/12F5Ia8Fps8kySRGrU35+9B0+Q3o/i7zEJLM2ufbLCfd8NfCIirstGW+si4o56D5i9YxcAfwEg6cXAXOCSeteamWEFMFocFRGzImLPiHhf6Z/4zsL3XUi9fgAyR/HDpB5hK/w5acRxh6SrJR1U5bwrga0kLciUzRjwtezYJ4F1wHck/UzSiU3Uf0/23NtHxFhEfDkrn/ac2fddCvuPRsQvaxxviOx5rpT0oKTHSKOkimaaCpRlzOUo/hb3Fb7/CnhujftVe6Y5wFbAqszMth74Vlae82BE/LrGvXcHflrjeC3OAd6aKaK3AxcWR5emM1gBmJxib/oe0jAfAEnPIZkf7m7yPqkg9QiPJJkovk7qpW56YcTG7Nhbsu2SrCdKRDwRESdExAtIpoUlkg5tQJ5aTHtOYI+sLGe77NkrHf8lqcHM2alGPV8imTl2j4htSX4SZcfqjWLKMuZyNPJbVKLaMz0EPAm8OFOWsyJi28x5nlNP1juBFzYgQ6V35FrgKeCPSCO/LzZwHzNDrABMJc4H3ilpLHOK/iOwIiJub+Da+4HdJG0JIGlLpbj2bSPit8DjJFNCNb4EvIlknsnNP0h6naS9sh7iY6Rw1lr3aYTzgb9XcjbPJtm7/710zsnZM/wR8DrgP7LyNcCfSdpKKd7/L2vUszXwSET8WtJ8UgOX82D2HC+ocu2lwIskvVXSMyS9CdiHmZlHNnmmiJgEPgucKul5AJJ2lfSaJu77OeADkg5QYi9JZeUF6R2Zq02d+OcCnwF+GxEVHeqmvVgBmE2IiO8CHwG+AtxL6tW9ueZFU1xBChG9T9JDWdnbgdslPU4yf7yt2sURsYLUu96FFKmUszfwXeAXwDXAv0bElQBKE9E+1KB8Rf4fsBJYS3IKr87Kcu4jRQzdA5wHvDcibs2OnUrqsd5PMl+cV6Oe9wEflfQESck8PQKKiF8B/wD8MDO9HFi8MCIeJjXSJ5DMcH8LvC4iHqI1aj3TB0lmtmuz3+q7QMNO74j4j+xZvgQ8QRrtbV/h1FyJPiyp6A/6IilIoKyETYfwRDBjKqA0gezfI2K3HovSNvr9mbKQ0weA/SPitl7LMwp4BGCM6Rf+GrjOjX/38Gw+Y0zPUUpVItJ8FNMlbAIyxpgRxSYgY4wZUQbCBDR79uyYO3dur8UwxpiBYtWqVQ9FxJxqxzumALJ8M+eS8rYEcGZEnK6UCOs9pPhngA9FxKW17jV37lxWrlzZKVGNMWYokVQzFUcnRwAbgBMiYrWkrUlTzC/Ljp0aEf/cwbqNMcbUoWMKICLuJU0iIiKekHQLreeSMcYY02a64gTOEnu9FFiRFR0vaa2ksyVtV+Wa4yStlLTywQcfrHSKMcaYGdBxBSDpuaSUAosj4nHgDFJqgTHSCOFTla6LiDMjYl5EzJszp6oPwxhjTIt0VAFI2oLU+J8XEV8FiIj7I2JjIfnU/E7KYIwxA0l5jlYH5mx1TAFkWRvPAm6JiGWF8p0Lp70euLFTMhhjzEAyMQHvf/9Uox+R9icm2lpNJ0cAf0jKAvkqSWuy7XDgE5JukLQWeCXw/g7KYIwxg0UErF8Pp58+pQTe//60v359W0cCnYwC+gFTi14UqRnzb4wxI40Ep56avp9+etoAFi1K5arUrLZY1SDkApo3b154IpgxZqSIgM0KRprJyaYbf0mrImJetePOBWSMMf1GbvYpUvQJtAkrAGOM6SeKNv9Fi1LPf9Gi6T6BNjEQyeCMMWZkkGDWrOk2/9wnMGuWfQDGGDP0RExv7Mv7DWAfgDHGDCLlxr6NPf8cKwBjjBlRrACMMYNFF1IkjApWAMaYwaFLKRJGBSsAY8xg0MUUCaOCw0CNMYNBF1MkjAoOAzXGDBZtSJEwKjgM1BgzPHQpRcKoYAVgjBkMupgiYVSwD8AYMxh0I0VCG2bfDhL2ARhjBotONdITEymaKFcu+Yhj1qyBDTO1D8AYM1x0IkXCiIaY2gRkjDEjGmJqE5AxxuQMWYipTUDGGNMItUJMB6Cj3Ao2ARljTNHmv2BB2iDt543/dtsNrDO4Gh4BGGNMHmI6Pp4a/+XLU/n4OKxYkfaH0BnsEYAxxkDq3ecNvDTlCIahdQbbCWyMMWWGxBlsJ7AxxjTDCOUbsgIwxpicEcs3ZB+AMcbkdCPfUB9hH4AxxpQZkqRwPfMBSNpd0pWSbpZ0k6RFWfn2ki6TdFv2uV2nZDDGmJboRL6hPqSTPoANwAkRsQ9wIPA3kvYBTgQuj4i9gcuzfWOMMV2mYwogIu6NiNXZ9yeAW4BdgSOBc7LTzgGO6pQMxhhjqtOVKCBJc4GXAiuAHSPi3uzQfcCOVa45TtJKSSsffPDBbohpjDEjRccVgKTnAl8BFkfE48VjkTzQFb3QEXFmRMyLiHlz5szptJjGGDNydFQBSNqC1PifFxFfzYrvl7Rzdnxn4IFOymCMMaYynYwCEnAWcEtELCscuhg4Jvt+DHBRp2QwxhhTnU5OBPtD4O3ADZLWZGUfAk4BLpT0l8AdwNEdlMEYY0wVOqYAIuIHQLXg2UM7Va8xxpjGcC4gY4wZUawAjDFmRLECMMaYEcUKwBhjRhQrAGOMGVGsAIwxpt2U0+z3adp9KwBjjGknExPTVw/LVxmbmOilVBWxAjDGmHYRAevXT19CMl9icv36vhsJeEnIRhiS1YGMMR2muITk6aenDaYvMdlHeARQjwEazhlj+oCiEsjpw8YfrABqM2DDOWNMH5C3E0WKncg+wgqgFrkmX7QoNfqbbZY++3Q4Z4zpMcVO4qJFMDk51X70oRKwAqjHAA3njBkJ+jnEUoJZs6Z3EvNO5KxZfddu2Alcj2rDOSsBY7rPxEQyv+b/f/n/56xZ/eOXm5iYHiiSK4E+bC88AqjFgA3njBlqBsknV27s+7DxB48AalNtOAd9OZwzZqgZsBDLQUDRT1qzCvPmzYuVK1f2TgDPAzCmf4hIARk5k5P+f6yCpFURMa/acZuAGmFAhnPGDD0DFGI5CFgBGGMGA/vk2o59AMaYwcA+ubZjH4AxZrCwT65h7AMwxgwX9sm1DSsAY4wZUWoqAEnbSjpF0q2SHpH0sKRbsrJZXZLRmMGln9MWmJGn3gjgQuBR4JCI2D4idgBemZVd2GnhjBlonErc9Dn1FMDciPh4RNyXF0TEfRHxcWDPzopmzAAzSGkLzMhSLwz0Dkl/C5wTEfcDSNoROBa4s8OyGTO4OG2BGQDqjQDeBOwAXC3pUUmPAlcB2wNH17pQ0tmSHpB0Y6FsQtLdktZk2+EzlN+Y/sWpxE2fU1MBRMSjEfHBiPjdiNgu234vK3ukzr2/ABxWofzUiBjLtktbFdyYvsdpC0yfUzcMVNJrJJ0h6eJsO0NSpYZ9GhHxPaCekjCmf5lJBI/TFpgBoKYPQNJpwIuAc4G7suLdgHFJr42IRS3UebykdwArgRMi4tEW7mFMZ5npwiNOW2AGgHpO4MMj4kXlQkkXAP8DNKsAzgA+BkT2+SngXZVOlHQccBzAHnvs0WQ1xsyAYgQPpIa72JtvNPXAAK0MZUaTegrg15L+ICKuK5X/AfDrZivLI4kAJH0WuKTGuWcCZ0LKBdRsXca0TDsjeJy2wPQx9RTAscAZkrZmygS0O/BYdqwpJO0cEfdmu68Hbqx1vjE9I1cCeeMP7r2boaOmAoiI1cACSTsBu2bFdxcnhlVD0vnAIcBsSXcBS4FDJI2RTEC3A3/VsuTGdJJqETxWAmaIqLsegKRtgVdQUACSvh0R62tdFxFvqVB8VtMSGtNtyhE8RR8AWAmYoaFeMrh3AKtJPfmtsu2VwKrsmOk3nHxs5lSL4Fm0yBE8ZqiouSCMpJ8AC8q9fUnbASsqRQh1Ai8I0yAzDV000/HCI2bAmemCMCLZ68tMZsdMv+DkY61Ra8TkCB4z5NTzAfwDsFrSd5hK/rYH8H9IcfymX3DysebxiMmMOPVyAZ0DzAOuBn6TbVcB8yLiC50WzjSJk481jkdMxtSPAspSNXy5C7KYmeLQxcbxiMmY1tYElvRdSd+U9Lp2C2RaxMnHmscjJjPi1B0BVOEdwM7AgW2UxcwEJx9rHo+YzIjTsAKQtD1ARDwSEfcA9wCrOiWYaQEnH2scT/Yypm466D2ATwCHAutTkbYBrgBOjIjbOy2gaRKHLjaGR0zG1J0Idg1wGvCfEbExK9sceCOwOCK6YgLyRDDTMTzZywwxM50INjsiLsgbf4CI2BgRXyatFWzMYOMRkxlh6vkAVkn6V+AcpiaC7Q4cA/y4k4KZLuJecH/h38N0iXojgHcANwAnA9/OtglSHv+3d1Qy0zrNJISbmJgeJpo7Rz0Ttjf49zBdpN5M4Kci4oyIOCwifj/bXhsR/xoRv+mWkANPNzJ05vfMG5DJyanyag2IZ8P2F/49TJdpdR4Akk6KiI+2U5ihpBv5ZvI6li2bakCuvhqOOAIee6z6WraeDdtf+Pcw3SYiWtqAn7d6bbPbAQccEAPJ5GTEokURkD4r7be7jo0bI8bG0n6+1atrcnL6+e2Qy7SOfw/TJoCVUasdr3kQHq+yPQFsqHVtO7eBVQAR0xvoRhvkdtTRaAPSDflM4/j3MG1kpgrg58COVY7dWevadm4DrQAiutOjK9fRSAPSjRGKaRz/HqbN1FMA9XwA5wJ7AvdXOPalGVqfRoPoQr6ZSnWMjcGqVbBkSfX0Bp4N21/49zDdppZ26JdtYEcAvfABnHTSlA8g9wksWhSxdGnte9TaN93Fv4dpE8xwBLAJWX6grSLi1varoyGjGz26SnUsXZp6/rNmwWab1R9teDZsf+Hfw3SJmrmAACSdApwbETdL+nNgGSkx3CUR8eHOizgEuYCiCzM7u1GHMWagmGkuIIDDIuLm7Pv7gVcD+wNeDKZRutGjc6/RGNMk9dJBLwV2lHQS8GzghcCbAAHbZuVXRcT3Oi6pMcaYtlJTAUTEyZL2IUUCbUMyBX1U0pbAq8MzgY0xZmBpxAn8LlJSuKdIYaEAewD/1CmhjDHGdJ66CiAifgmcUSpbB6zrlFDGdBU70M2I0ogTuCUknS3pAUk3Fsq2l3SZpNuyz+06Vb8xDeH0y2aE6ZgCAL4AHFYqOxG4PCL2Bi7P9o3pDeH0y2a0aTkddD0i4nuS5paKjwQOyb6fA1wFfLBTMhhTE6dfNiNO3YlgAJLmAO8B5lJQGhHxrjrXzSVNGHtJtr8+ImZl3wU8mu9XuPY44DiAPfbY44A77rijrpzGtEREmjGdMznpxt8MBe2YCAZwEbAt8F3gG4WtZbI8FVW1T0ScGRHzImLenDlzZlKVMdWplqzP5h8zAjRqAtoqItphqrlf0s4Rca+knYEH2nBPY1qjaPPPzT75PtgMZIaeRhXAJZIOj4hLZ1jfxcAxwCnZ50UzvN/McPjfaNNMsj6/K2YIadQH8ATwHOA3wG9JqSAiIrapcc35JIfvbNJ6AkuBrwMXkiaS3QEcHRGP1Ku/I8ngurFWrxkM6jXuflfMgFLPB9DQCCAitm624oh4S5VDhzZ7r7ZTDP+D6UP/Sounm+GmViI9vytmiKk5ApD0uxFxq6T9Kx2PiNUdk6xAR0YARftvjsP/TCX8rpgBpd4IoJ4CODMijpN0ZYXDERGvaoeQ9ejYegAO/zON4nfFDCAzCgONiOOyz1dW2LrS+HcMh/+ZRvG7YoaUTqaC6F/K4X+Tk+mzmBLAGPC7YoaajqWC6Gu6sVavGQ78rpghpqEw0F7TUR+AY7sHi179Zn5XzADSllQQki5vpGzg8Dq6g0UvUzf7XTFDSE0FIOlZkrYHZkvaLsvnv32W5G3XrkhoBoPySLLdI0unbjam7dTzAfwVsBjYBSjG/D8OfKZDMplBoxszZZ262Zi2Uy8M9PSIeD7wgYh4fmHbLyKsAEaJaj38bvbMi0ogx42/MS1TcwQg6VURcQVwt6Q/Kx+PiK92TDLTP9Tr4XerZ14tHt9KwJiWqOcEfnn2+acVttd1UK7e0ml7difolMyN9PC70TN3PL4xbaeeD+DR7POsiPhBp4XpCwYx82MnZW7E9t5qz7yZ0ErH4xvTfiKi6gasyT5X1zqv09sBBxwQXWFyMmLRoghIn5X2+41uyTw5me6Zb/l9W61/6dLpx/Prli6tL0etfWPM0wAro0bbWm8EcIuk24BdJK0tlOfrAezbEa3UK/JeZcT03u74eHfszK1MNpppdEwjddbr4ec982XLpsuz7baV659JimXH4xvTPmpph6RA2Am4HtizvNW7tl1b10YAEakHOj4+vbc7Pl6/Z9qOeiv1iE86afp51Xq81XrordRZfNZGeviTk0nO4v7GjbV79MX75Fu/jrKMGVCoMwKoOxM4Iu6LiP2Ae4Gts+2eiLijQzqpd0TAo4/C8uXTy5cvT+WVHI3lslackcUecdnRevHFyeGZn1ec+ZrXVa2HXkuWWnUWwzer2d4XLUrlJ58MixdP3Wvx4rQdfHDtUFCHdBrTe2pph3wDXkFawvFq4HvA/wIvb+Tadmxd9QGUe//FUUC5d9qqHbta3eUe8dhY9Z533uPOe9r5+Xl5Iz3qZnrhlWzvxevHxyuPnGqNWDwCMKajUGcE0KgCWAX8TmH/RcCqRq5tx9aXJqBOOF/LZpxi415sJIvluTIoKot65pdadTYrd6WGvN69BtHZbswA0i4FsLaRsk5tXVUAlUYB1Xqy7ezFVrvXxo2VG9V6dTciQ7vkLyuRRu7VztGTMaYi7VIAnwc+BxySbZ8Fzm7k2nZsfR0GWis8sh315j37ao18q733dvXCqynMvKzWvRzSaUxHqacAGl0R7L3AzcB4tt0M/HXrnoc+pZ7Ds9HwyMnJ5tIUV6p32TIYG4M1ayrPfM3rKNcdNRy/M3nWSuTPv3w5LFiQwmXHx6ec6OPjte/lkE5jekst7ZAUCJsDt9Y7r5NbSyOAmfQuG7m2Ui+9/NmsOaV8bjG0slhn0dGbm6fK+63W2WwvvGjKKTqGly51j96YHkOdEUBDK4JJughYGBE/76w6qkzTK4J1K51DXs+yZbBkydTEJmhfMrSIyhO1JibgW99KPe/TTkvHFi+GFSvgsMO6m7aimozGmJ7SlhXBgO2AmyRdLunifGuPiG0mojvpiSOmMmFutllSAkXKjX+53mZMNZX2ly5Njf/y5dNNQStWzPw5m5W126acVv+WxphpNLoo/Ec6KkU76cbCIRMTaWLYaael+23cCPNKSraYKqETIxJpqv52Pme/J8Prd/mMGSRq2YeAZ5FWBPsMaXWwZ9Q6v1Nbyz6AcoRMO2zSk5MRCxZM2dtPOili9uy0v9NOERs2VI7J71TMeyORQI3a+fs9Pr/f5TOmz2AmYaDABcC/Z43/14HTa53fqa1pBVBspMuhiTONMz/ppIj99pt+73w7/vip8Md8Vm4uz0xm3NZ6znr3bTbevt9n6Pa7fMb0ETNVADcUvj+DNqWFBm4HbgDW1BMwmlUA5bj0coqCZqNkyvfOG59qSqDY8y9fW62nnn9vprFuNElbKz3mmc4O7jT9Lp8xfcJMFcDqWvutbpkCmN3o+U2PAPJ0DuUJSgsWtMfkUi1fULUGqVKvNVdElUI7G22sm83m2UiPud972P0unzF9xEwVwEbg8Wx7AthQ+P54rWvr3LezCiBiqnFtpKfYrNmlngKo1ChXG40UZ8xWy/1TT556sjfzd+hnG3u/y2dMnzEjBdCpjZRNdDUpydxxVc45DlgJrNxjjz2af/JGe4rN2sg3bozYccfKDf/8+ZVTIJQnS5UVSCPpHVqdsNVsj7nfc/T0u3zG9BH9qgB2zT6fR1ps5uW1zm/JCdxIT7HZHmXZB7Bx45Szed99kxmnWoNUtvmXG/myzb7YWFebEdzI8omt+gBq7feafpfPmD6hngJodB5AW4mIu7PPByR9DZhPWmegPTSygHhE83MGyssfbrYZXHNNmoGbL44C1a9ND71pDp+DDoL589M5y5enHDqQJnWdfvpUTqD83rWWTyzuS2lZxmYXUu/3HD39Lp8xA0JDqSDaWqH0HGCziHgi+34Z8NGI+Fa1a5pOBZFTrXEsTyaanITNN586b3KydqNSq9GtJ0+x8T711NT4r1iRju+yCzzvean+tWth4UL4/vdhm23gpS+tnmqi2nPl9W277ZRyakZeY8xA065UEO1kR+AHkq4HfgR8o1bjPyMq9RQjpqeKmJyEAw6Yfl69rJqN9kAr3aM4MolI6RwgNfz33JN6+mvXwn77pfuuWZMa/3KqiXwB9ryRX7q0egqMxx6bLku1hdrryW6MGS5q2Yf6ZWt5PYBqtuJKtvaxscqzdlullrNycnLq+MaNtaOKqkUHVZK3lSiierIaYwYW+tEJ3OzWkgKo16iVHbH5xK1mGr9GFEwl52u54d6woboC+MhHNr22nYvEOLTSmKFlNBVArUZtfLx+T7mRRq8RBVOvjvLxStvY2JQSyK9tdZnIRv5ezVxnjOlrRlMBRFRu1BYsiFi4cMrkMj6e9vNQzkopHOrdu16Yaa3eeLkhL/fsd9ppU7mqNdYzTTrn9ArGDB31FEAvnMDdoRjymLNgAXz60ynqZnwcrr027c+fn/a33TYt7DIxUd8JnC+fePrpKRy0GNlTdM4WyZ2z+bZkyfTjW245ff/uu6eWaNxss02jiIrLRC5ZUjnks5ElHmvJaowZXmpph37Z2jYCqJQfqNrIIE/1XL5nkUZMMeXe+IIF0+XIv8+fv6lclUYkjZieaslc6+9kH4AxQwUjaQKq5wOoZXYZG5tqjIspncsNbz1nbKWGenx8ytxUVgT5vfKEdfVmJNfabxZHARkzlIymAoiYyghabNSK9v56W57uuZJ9vdj41wofrdRQ10r3UJa3m42w0ysYM3TUUwBdnwncCi3NBC4v25hP+FqzZspOvmhR8gFUo5JNvzgbd2wMVq2abp9vZGnCiHRNTj7zOMKLqxtj2kY/zgTuPBFpVmy+YHpkDtc1a1Kjnc+qrde45rNt83PLTuW88S8eb6Txr+ZwdY4bY0wXGU4FUCtKJ2+088RuY2PV77P//ql3DpUb7iVLpo7n9daiOIooR/E46sYY02WGUwFA5R77qadO9dgjUo6cNWtSeOiOO06dNz4O++4L118PBx+cGuq84R4bg40bpxruAw5IeXgalalSltJGQjWNMabNDK8CqGVqgemN8X//N8ydm8rHxlL5IYdMXZenVc7TMi9ZksxD+X452VotJiamp4tu1HRkjDHtppaHuF+2GYWB5pE15f3iuflnOUS0fG6rydaMMaYHMJIzgfPefZ5qGVIve+HCNAs4z41fdLxKKWKoSB5BlLPZZpXNSjbdGGMGkOFUAJDs8gsWTEUCTUzA976XFMD69cmuv3jxlOklt/MXydcLyM079cxKxhgzQPRkSciukPfopU1j95ctSw338uVJSUxOwn/9V7Ln77QTvOEN8IMfpOuuvjrl6DnssGTrL+b8Kc4L8EjAGDNgDO9EsJzypKsyCxdOrcU7ezY89NDUseL++HgyKz322KaTwxqZ/GWMMV2m3kSw4R0BQGWTTZHx8Sm7f3mkANMb/3w0UVSYlUJNjTFmQBheH0B50tXGjdUnfdVryIvO4ImJTe3+uY/BGGMGiOFVAMU4/9zmv2bN9HOWL0+O4EoO4CKLFqUGf3ISLr648sLr69fXdwaXjw+A+c0YM7yMhg8A4KCDUgTQwoWpwV68OCmA+fPhwAPT9/32S7N/c4o+gIUL0+enPz01ASynOLO3GhMTSUnYf2CM6RKj7QOAqUb5sMNSxE9uzlmzBnbYAV7zmuQkHh+HCy6ArbeGY4+FH/0oNdK77pqUQp41dMGCNHN4882n6qjX+OfJ6YoRQ0XzlLN+GmN6Qa1ZYv2ytbQeQCUqLeYyNhaxYcNU/v+xsYinnpq+Yle9VcUaWUvYC68bY7oMI7seQE65d503vzC1PkDO2BisXAknnJBy/+QppYvkZqJ8LYAlS6aSxB1xxNQs42qyVFoHwBhjOsBorgeQU47YiYDdd09mHUiNfZHrrkuNf+7ULaZ6hmQmuv765BtoNimcZxEbY/qM4fUBVLK7L1oEd9+d9vfff9NrttgifS5aBJ/6FOy226bnjI+n0cHjj6d75/cvrx5WHnUUbf6eRWyM6Qdq2Yf6ZWvZB1DJ7r5wYcS++04vK+8/9VT1NX/zDKGTk9OvyX0A1dby7dTC617L1xhTBer4AIZ3BABTefyLnHZaCgFdu3aq7BWvmL6/5Zbps7jmbz5RbNas9Fk25xxwwHSfQDm6Z2Ji0+yjM+35O7TUGDMTammHTm3AYcBPgHXAifXOb3kEUIz2ybc82qe8VSrfsGH6/fKefz4ayKN/ynV0I7qnLEelfWPMSEOdEUAvGv/NgZ8CLwC2BK4H9ql1TUsKoNgglhvovMFfuLC2EqjWkJbNORs3Tr+uW42vQ0uNMTWopwB6EQU0H1gXET+LiKeALwNHtr2WYiqIVaumH9txR1i9etPkb9dfP5U3qNZi7cVlHSOS2adIt6J7qq17bIeyMaYBeqEAdgXuLOzflZVNQ9JxklZKWvnggw+2VtPERArTLDfQRx+dPst2/HytgNzmX2ux9qLNPbf5T07WVhztJq+/iENLjTEN0rfzACLizIiYFxHz5syZ0+pNpjtl8wb6059OTttyeR7bH9HYYu3FUUbe866nONpFr5WPMWbg6UUU0N3A7oX93bKy9pM30OPj0xvoiClzT7EcpjfcjTTgnYjuaYRqygc6r3yMMUNBLxTAdcDekp5PavjfDLy161IcckhaN7gdDXf5mm41vr1SPsaYoaDrJqCI2AAcD3wbuAW4MCJu6lBlU/l8ivn7ly9P5WUGseHslfIxxgw8PZkIFhGXApd2vKKiWaSctmHZMjeWxpiRpm+dwG2j0mzgPDLIs2WNMSPM8CuAfBnHInkEUCPLOFajfJ2jbowxA8ZwK4A8DHTNmukLwuf7rZqBKqWZ9sLwxpgBY7gVQK3ZwEccMX1xlkbJHcutLgxvjDF9wnBnA4XUK5+c3HQ2cL54S7MjgFqOZYdgGmMGiOEeAUD12cAzmTHrHDzGmCFg+BVAJ9I1OAePMWYIGH4TELR3xmw5B4+XdzTGDCijoQCgfTNmnYPHGDMkKAbAbDFv3rxYuXJlr8WYTtmB3IpD2RhjOoikVRExr9rx4fUBdHqilnPwGGMGnOFUAJ6oZYwxdRk+BeCJWsYY0xDD5wT2RC1jjGmI4XUCR0xP9TA56cbfGDNSjKYT2BO1jDGmLsOnALxYujHGNMRw+gA8UcsYY+oy3D4AT9Qyxowwo+kDAE/UMsaYOgyvAjDGGFMTKwBjjBlRrACMMWZEsQIwxpgRZSCigCQ9CNzR4uWzgYfaKE6nsbydZ9BktrydZdDkhcZl3jMi5lQ7OBAKYCZIWlkrDKrfsLydZ9BktrydZdDkhfbJbBOQMcaMKFYAxhgzooyCAjiz1wI0ieXtPIMms+XtLIMmL7RJ5qH3ARhjjKnMKIwAjDHGVMAKwBhjRpShUQCSbpd0g6Q1kjZJHarEcknrJK2VtH8v5Mxk+Z1Mznx7XNLi0jmHSHqscM5JXZbxbEkPSLqxULa9pMsk3ZZ9blfl2mOyc26TdEyPZf6kpFuz3/xrkmZVubbm+9NFeSck3V343Q+vcu1hkn6Svc8n9lDeCwqy3i5pTZVre/H33V3SlZJulnSTpEVZeV++xzXk7dw7HBFDsQG3A7NrHD8c+CYg4EBgRa9lzuTaHLiPNGGjWH4IcEkP5Xo5sD9wY6HsE8CJ2fcTgY9XuG574GfZ53bZ9+16KPOrgWdk3z9eSeZG3p8uyjsBfKCBd+anwAuALYHrgX16IW/p+KeAk/ro77szsH/2fWvgf4B9+vU9riFvx97hoRkBNMCRwLmRuBaYJWnnXgsFHAr8NCJanencESLie8AjpeIjgXOy7+cAR1W49DXAZRHxSEQ8ClwGHNYpOYtUkjkivhMRG7Lda4HduiFLI1T5GzfCfGBdRPwsIp4Cvkz6bTpKLXklCTgaOL/TcjRKRNwbEauz708AtwC70qfvcTV5O/kOD5MCCOA7klZJOq7C8V2BOwv7d2VlvebNVP+nOUjS9ZK+KenF3RSqCjtGxL3Z9/uAHSuc069/Z4B3kUaBlaj3/nST47Ph/tlVzBP9+Df+I+D+iLityvGe/n0lzQVeCqxgAN7jkrxF2voOD9OSkC+LiLslPQ+4TNKtWY+lb5G0JXAE8HcVDq8mmYV+kdmBvw7s3UXxahIRIWlgYoglfRjYAJxX5ZR+eX/OAD5G+mf+GMms8q4eyNEsb6F2779nf19JzwW+AiyOiMdVWByqH9/jsryF8ra/w0MzAoiIu7PPB4CvkYbJRe4Gdi/s75aV9ZLXAqsj4v7ygYh4PCJ+kX2/FNhC0uxuC1ji/txsln0+UOGcvvs7SzoWeB3wtsiMpWUaeH+6QkTcHxEbI2IS+GwVOfrqbyzpGcCfARdUO6dXf19JW5Aa0/Mi4qtZcd++x1Xk7dg7PBQKQNJzJG2dfyc5TW4snXYx8A4lDgQeKwwDe0XVXpOknTK7KpLmk36rh7soWyUuBvJoiGOAiyqc823g1ZK2y8wXr87KeoKkw4C/BY6IiF9VOaeR96crlPxSr68ix3XA3pKen40i30z6bXrFHwO3RsRdlQ726u+b/f+cBdwSEcsKh/ryPa4mb0ff4U56tbu1kaIhrs+2m4APZ+XvBd6bfRfwL6ToiRuAeT2W+TmkBn3bQllR3uOzZ7me5Pg5uMvynQ/cC/yWZP/8S2AH4HLgNuC7wPbZufOAzxWufRewLtve2WOZ15FsuWuy7d+yc3cBLq31/vRI3i9m7+daUkO1c1nebP9wUpTIT3spb1b+hfy9LZzbD3/fl5FMaWsLv//h/foe15C3Y++wU0EYY8yIMhQmIGOMMc1jBWCMMSOKFYAxxowoVgDGGDOiWAEYY8yIYgVgGkLSxizL4I2S/kPSVm2+/1WSai5yLWlxsV5Jl1bLjNgmmeZIWiHpx5L+qHRsC0mnZJkiV0u6RtJri3Jl2/uarHMXSf/Z5DXHK2UFjeJkwWzOS90MuKqSWTSbZ7AiK78gm3OApGdm++uy43Obkdf0D1YAplGejIixiHgJ8BRpzkK3WQw8rQAi4vCIWN/B+g4FboiIl0bE90vHPkbK3viSiNiflFBs65Jcs4CmFEBE3BMRb2hSzh+SJmOVEwq+lpQ+ZG/gOFKaiWlI2pw0P+a1pMyTb5G0T3b448CpEbEX8ChpngLZ56NZ+anZeWYAsQIwrfB9YC+lvOpfz3qX10raF57Oaf/FrFd8m6T3ZOWHSLokv4mkz2RT3Kch6QxJK5Vyop+clY2TJr5cKenKrOz2vMcraUk2OrlR2doKkuZKukXSZ7N7fUfSsyvUN1fSFdlzXC5pD0ljpLTBR2Yjn2cXzt8KeA+wMCJ+A0+ncLiwJNcpwAuz6z8p6VxJRxXuc56kaVk8M1luzL4fK+mrkr6V/R0/UenHiIgfR8TtFQ41kgG3YmbRbFbqq4B8NFLMmlnMpvmfwKHZaOPFkn6UPe9aSX2Tu8pUxgrANIVS3pfXkmarngz8OCL2BT4EnFs4dV9SA3IQcJKkXZqo5sMRMS+7xysk7RsRy4F7gFdGxCtLMh0AvBNYQFrr4T2SXpod3hv4l4h4MbAe+PMK9X0aOCd7jvOA5RGxBjgJuCAb+TxZOH8v4OdRSNRVhRNJqb7HIuL/kqb5H5vJvC1wMPCNOvcYA94E/D7wJkm71z59Go1ktKx2zg7A+phKQ1y89ulrsuOPZee/Fzg9IsZIs2orpoYw/YMVgGmUZyut9rQS+DmpMXsZKXUBEXEFsIOkbbLzL4qIJyPiIeBKmkv+dbSk1cCPgReTTBO1eBnwtYj4ZaQEel8lpScG+N+sMQdYBcytcP1BwJey71/M7td2IuJqUg6fOaQ8UF8pNLDVuDwiHouIXwM3A3t2QrY2cQ3wIUkfJGWyfbLeBaa3WAGYRsl9AGMRsTAzF9SinGMkSKlsi+/cs8oXSXo+8AHg0KxH/o1K5zXBbwrfN9KeFOjrgD0Kyq4ZzgX+gjRiObuB82cifyMZLaud8zDJZPSMCtc+fU12fFvg4Yj4Eim9+ZPApZJe1YSspgdYAZiZ8H3gbZDs+8BDBbPIkZKeJWkH0vKW15GclPtkUSSzSE7WMtsAvwQek7QjydyU8wSZo7WCHEdJ2kopE+Lrs7JG+W9SRk2y56l5baSMjGcBpxciY+ZIemPp1EryfoHkzCYibm5CxlaomgFX0q3ZORUzi0ZKEnYlkDuki1kzi9k03wBcEREh6QXAzzJz3UUkE57pY6wAzEyYAA6QtJbk8CwunL2W1IBcC3wsi265E7iQlKb2QpKJZxoRcX1WfivJLPPDwuEzgW/lTuDCNatJDeuPSCsofS4iNrl3DRYC78ye4+3Aogau+XvgQeDmzGl7CTDNJxARDwM/zBzTn8zK7ict9ff5JuSriaRxSXeReulrJX0uO3QpaS3bdaS1Bd6XnT+blB03t+EfT0p1fAtwYUTclF3/QWCJpHUkG/9ZWflZJHPfOmAJydcBaUnIGzNT4UuY7hMyfYizgZq2I2kC+EVE/HOvZek3sgiiG0iLfz/WIxleB7wg66mbEWaYloQ0pq+R9Mek3vOpvWr8ASLikvpnmVHAIwBjjBlR7AMwxpgRxQrAGGNGFCsAY4wZUawAjDFmRLECMMaYEeX/AytMvVQB4LocAAAAAElFTkSuQmCC\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "# Create a scatter plot of the data. To change the markers to red \"x\",\n",
    "# we used the 'marker' and 'c' parameters\n",
    "plt.scatter(x_train, y_train, marker='x', c='r') \n",
    "\n",
    "# Set the title\n",
    "plt.title(\"Profits vs. Population per city\")\n",
    "# Set the y-axis label\n",
    "plt.ylabel('Profit in $10,000')\n",
    "# Set the x-axis label\n",
    "plt.xlabel('Population of City in 10,000s')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Your goal is to build a linear regression model to fit this data.\n",
    "- With this model, you can then input a new city's population, and have the model estimate your restaurant's potential monthly profits for that city."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<a name=\"4\"></a>\n",
    "## 4 - Refresher on linear regression\n",
    "\n",
    "In this practice lab, you will fit the linear regression parameters $(w,b)$ to your dataset.\n",
    "- The model function for linear regression, which is a function that maps from `x` (city population) to `y` (your restaurant's monthly profit for that city) is represented as \n",
    "    $$f_{w,b}(x) = wx + b$$\n",
    "    \n",
    "\n",
    "- To train a linear regression model, you want to find the best $(w,b)$ parameters that fit your dataset.  \n",
    "\n",
    "    - To compare how one choice of $(w,b)$ is better or worse than another choice, you can evaluate it with a cost function $J(w,b)$\n",
    "      - $J$ is a function of $(w,b)$. That is, the value of the cost $J(w,b)$ depends on the value of $(w,b)$.\n",
    "  \n",
    "    - The choice of $(w,b)$ that fits your data the best is the one that has the smallest cost $J(w,b)$.\n",
    "\n",
    "\n",
    "- To find the values $(w,b)$ that gets the smallest possible cost $J(w,b)$, you can use a method called **gradient descent**. \n",
    "  - With each step of gradient descent, your parameters $(w,b)$ come closer to the optimal values that will achieve the lowest cost $J(w,b)$.\n",
    "  \n",
    "\n",
    "- The trained linear regression model can then take the input feature $x$ (city population) and output a prediction $f_{w,b}(x)$ (predicted monthly profit for a restaurant in that city)."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<a name=\"5\"></a>\n",
    "## 5 - Compute Cost\n",
    "\n",
    "Gradient descent involves repeated steps to adjust the value of your parameter $(w,b)$ to gradually get a smaller and smaller cost $J(w,b)$.\n",
    "- At each step of gradient descent, it will be helpful for you to monitor your progress by computing the cost $J(w,b)$ as $(w,b)$ gets updated. \n",
    "- In this section, you will implement a function to calculate $J(w,b)$ so that you can check the progress of your gradient descent implementation.\n",
    "\n",
    "#### Cost function\n",
    "As you may recall from the lecture, for one variable, the cost function for linear regression $J(w,b)$ is defined as\n",
    "\n",
    "$$J(w,b) = \\frac{1}{2m} \\sum\\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2$$ \n",
    "\n",
    "- You can think of $f_{w,b}(x^{(i)})$ as the model's prediction of your restaurant's profit, as opposed to $y^{(i)}$, which is the actual profit that is recorded in the data.\n",
    "- $m$ is the number of training examples in the dataset\n",
    "\n",
    "#### Model prediction\n",
    "\n",
    "- For linear regression with one variable, the prediction of the model $f_{w,b}$ for an example $x^{(i)}$ is representented as:\n",
    "\n",
    "$$ f_{w,b}(x^{(i)}) = wx^{(i)} + b$$\n",
    "\n",
    "This is the equation for a line, with an intercept $b$ and a slope $w$\n",
    "\n",
    "#### Implementation\n",
    "\n",
    "Please complete the `compute_cost()` function below to compute the cost $J(w,b)$."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<a name=\"ex01\"></a>\n",
    "### Exercise 1\n",
    "\n",
    "Complete the `compute_cost` below to:\n",
    "\n",
    "* Iterate over the training examples, and for each example, compute:\n",
    "    * The prediction of the model for that example \n",
    "    $$\n",
    "    f_{wb}(x^{(i)}) =  wx^{(i)} + b \n",
    "    $$\n",
    "   \n",
    "    * The cost for that example  $$cost^{(i)} =  (f_{wb} - y^{(i)})^2$$\n",
    "    \n",
    "\n",
    "* Return the total cost over all examples\n",
    "$$J(\\mathbf{w},b) = \\frac{1}{2m} \\sum\\limits_{i = 0}^{m-1} cost^{(i)}$$\n",
    "  * Here, $m$ is the number of training examples and $\\sum$ is the summation operator\n",
    "\n",
    "If you get stuck, you can check out the hints presented after the cell below to help you with the implementation."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [],
   "source": [
    "# UNQ_C1\n",
    "# GRADED FUNCTION: compute_cost\n",
    "\n",
    "def compute_cost(x, y, w, b): \n",
    "    \"\"\"\n",
    "    Computes the cost function for linear regression.\n",
    "    \n",
    "    Args:\n",
    "        x (ndarray): Shape (m,) Input to the model (Population of cities) \n",
    "        y (ndarray): Shape (m,) Label (Actual profits for the cities)\n",
    "        w, b (scalar): Parameters of the model\n",
    "    \n",
    "    Returns\n",
    "        total_cost (float): The cost of using w,b as the parameters for linear regression\n",
    "               to fit the data points in x and y\n",
    "    \"\"\"\n",
    "    # number of training examples\n",
    "    m = x.shape[0] \n",
    "    \n",
    "    # You need to return this variable correctly\n",
    "    total_cost = 0\n",
    "    \n",
    "    ### START CODE HERE ###  \n",
    "\n",
    "    ### END CODE HERE ### \n",
    "\n",
    "    return total_cost"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<details>\n",
    "  <summary><font size=\"3\" color=\"darkgreen\"><b>Click for hints</b></font></summary>\n",
    "    \n",
    "    \n",
    "   * You can represent a summation operator eg: $h = \\sum\\limits_{i = 0}^{m-1} 2i$ in code as follows:\n",
    "     ```python \n",
    "    h = 0\n",
    "    for i in range(m):\n",
    "        h = h + 2*i\n",
    "    ```\n",
    "  \n",
    "   * In this case, you can iterate over all the examples in `x` using a for loop and add the `cost` from each iteration to a variable (`cost_sum`) initialized outside the loop.\n",
    "\n",
    "   * Then, you can return the `total_cost` as `cost_sum` divided by `2m`.\n",
    "     \n",
    "    <details>\n",
    "          <summary><font size=\"2\" color=\"darkblue\"><b> Click for more hints</b></font></summary>\n",
    "        \n",
    "    * Here's how you can structure the overall implementation for this function\n",
    "    ```python \n",
    "    def compute_cost(x, y, w, b):\n",
    "        # number of training examples\n",
    "        m = x.shape[0] \n",
    "    \n",
    "        # You need to return this variable correctly\n",
    "        total_cost = 0\n",
    "    \n",
    "        ### START CODE HERE ###  \n",
    "        # Variable to keep track of sum of cost from each example\n",
    "        cost_sum = 0\n",
    "    \n",
    "        # Loop over training examples\n",
    "        for i in range(m):\n",
    "            # Your code here to get the prediction f_wb for the ith example\n",
    "            f_wb = \n",
    "            # Your code here to get the cost associated with the ith example\n",
    "            cost = \n",
    "        \n",
    "            # Add to sum of cost for each example\n",
    "            cost_sum = cost_sum + cost \n",
    "\n",
    "        # Get the total cost as the sum divided by (2*m)\n",
    "        total_cost = (1 / (2 * m)) * cost_sum\n",
    "        ### END CODE HERE ### \n",
    "\n",
    "        return total_cost\n",
    "    ```\n",
    "    \n",
    "    If you're still stuck, you can check the hints presented below to figure out how to calculate `f_wb` and `cost`.\n",
    "    \n",
    "    <details>\n",
    "          <summary><font size=\"2\" color=\"darkblue\"><b>Hint to calculate f_wb</b></font></summary>\n",
    "           &emsp; &emsp; For scalars $a$, $b$ and $c$ (<code>x[i]</code>, <code>w</code> and <code>b</code> are all scalars), you can calculate the equation $h = ab + c$ in code as <code>h = a * b + c</code>\n",
    "          <details>\n",
    "              <summary><font size=\"2\" color=\"blue\"><b>&emsp; &emsp; More hints to calculate f</b></font></summary>\n",
    "               &emsp; &emsp; You can compute f_wb as <code>f_wb = w * x[i] + b </code>\n",
    "           </details>\n",
    "    </details>\n",
    "\n",
    "     <details>\n",
    "          <summary><font size=\"2\" color=\"darkblue\"><b>Hint to calculate cost</b></font></summary>\n",
    "          &emsp; &emsp; You can calculate the square of a variable z as z**2\n",
    "          <details>\n",
    "              <summary><font size=\"2\" color=\"blue\"><b>&emsp; &emsp; More hints to calculate cost</b></font></summary>\n",
    "              &emsp; &emsp; You can compute cost as <code>cost = (f_wb - y[i]) ** 2</code>\n",
    "          </details>\n",
    "    </details>\n",
    "        \n",
    "    </details>\n",
    "\n",
    "</details>\n",
    "\n",
    "    \n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "You can check if your implementation was correct by running the following test code:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "<class 'numpy.float64'>\n",
      "Cost at initial w (zeros): 75.203\n",
      "\u001b[92mAll tests passed!\n"
     ]
    }
   ],
   "source": [
    "# Compute cost with some initial values for paramaters w, b\n",
    "initial_w = 2\n",
    "initial_b = 1\n",
    "\n",
    "cost = compute_cost(x_train, y_train, initial_w, initial_b)\n",
    "print(type(cost))\n",
    "print(f'Cost at initial w (zeros): {cost:.3f}')\n",
    "\n",
    "# Public tests\n",
    "from public_tests import *\n",
    "compute_cost_test(compute_cost)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Expected Output**:\n",
    "<table>\n",
    "  <tr>\n",
    "    <td> <b>Cost at initial w (zeros):<b> 75.203 </td> \n",
    "  </tr>\n",
    "</table>"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<a name=\"6\"></a>\n",
    "## 6 - Gradient descent \n",
    "\n",
    "In this section, you will implement the gradient for parameters $w, b$ for linear regression. "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "As described in the lecture videos, the gradient descent algorithm is:\n",
    "\n",
    "$$\\begin{align*}& \\text{repeat until convergence:} \\; \\lbrace \\newline \\; & \\phantom {0000} b := b -  \\alpha \\frac{\\partial J(w,b)}{\\partial b} \\newline       \\; & \\phantom {0000} w := w -  \\alpha \\frac{\\partial J(w,b)}{\\partial w} \\tag{1}  \\; & \n",
    "\\newline & \\rbrace\\end{align*}$$\n",
    "\n",
    "where, parameters $w, b$ are both updated simultaniously and where  \n",
    "$$\n",
    "\\frac{\\partial J(w,b)}{\\partial b}  = \\frac{1}{m} \\sum\\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)}) \\tag{2}\n",
    "$$\n",
    "$$\n",
    "\\frac{\\partial J(w,b)}{\\partial w}  = \\frac{1}{m} \\sum\\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) -y^{(i)})x^{(i)} \\tag{3}\n",
    "$$\n",
    "* m is the number of training examples in the dataset\n",
    "\n",
    "    \n",
    "*  $f_{w,b}(x^{(i)})$ is the model's prediction, while $y^{(i)}$, is the target value\n",
    "\n",
    "\n",
    "You will implement a function called `compute_gradient` which calculates $\\frac{\\partial J(w)}{\\partial w}$, $\\frac{\\partial J(w)}{\\partial b}$ "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<a name=\"ex02\"></a>\n",
    "### Exercise 2\n",
    "\n",
    "Please complete the `compute_gradient` function to:\n",
    "\n",
    "* Iterate over the training examples, and for each example, compute:\n",
    "    * The prediction of the model for that example \n",
    "    $$\n",
    "    f_{wb}(x^{(i)}) =  wx^{(i)} + b \n",
    "    $$\n",
    "   \n",
    "    * The gradient for the parameters $w, b$ from that example \n",
    "        $$\n",
    "        \\frac{\\partial J(w,b)}{\\partial b}^{(i)}  =  (f_{w,b}(x^{(i)}) - y^{(i)}) \n",
    "        $$\n",
    "        $$\n",
    "        \\frac{\\partial J(w,b)}{\\partial w}^{(i)}  =  (f_{w,b}(x^{(i)}) -y^{(i)})x^{(i)} \n",
    "        $$\n",
    "    \n",
    "\n",
    "* Return the total gradient update from all the examples\n",
    "    $$\n",
    "    \\frac{\\partial J(w,b)}{\\partial b}  = \\frac{1}{m} \\sum\\limits_{i = 0}^{m-1} \\frac{\\partial J(w,b)}{\\partial b}^{(i)}\n",
    "    $$\n",
    "    \n",
    "    $$\n",
    "    \\frac{\\partial J(w,b)}{\\partial w}  = \\frac{1}{m} \\sum\\limits_{i = 0}^{m-1} \\frac{\\partial J(w,b)}{\\partial w}^{(i)} \n",
    "    $$\n",
    "  * Here, $m$ is the number of training examples and $\\sum$ is the summation operator\n",
    "\n",
    "If you get stuck, you can check out the hints presented after the cell below to help you with the implementation."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# UNQ_C2\n",
    "# GRADED FUNCTION: compute_gradient\n",
    "def compute_gradient(x, y, w, b): \n",
    "    \"\"\"\n",
    "    Computes the gradient for linear regression \n",
    "    Args:\n",
    "      x (ndarray): Shape (m,) Input to the model (Population of cities) \n",
    "      y (ndarray): Shape (m,) Label (Actual profits for the cities)\n",
    "      w, b (scalar): Parameters of the model  \n",
    "    Returns\n",
    "      dj_dw (scalar): The gradient of the cost w.r.t. the parameters w\n",
    "      dj_db (scalar): The gradient of the cost w.r.t. the parameter b     \n",
    "     \"\"\"\n",
    "    \n",
    "    # Number of training examples\n",
    "    m = x.shape[0]\n",
    "    \n",
    "    # You need to return the following variables correctly\n",
    "    dj_dw = 0\n",
    "    dj_db = 0\n",
    "    \n",
    "    ### START CODE HERE ### \n",
    "    \n",
    "    ### END CODE HERE ### \n",
    "        \n",
    "    return dj_dw, dj_db"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<details>\n",
    "  <summary><font size=\"3\" color=\"darkgreen\"><b>Click for hints</b></font></summary>\n",
    "       \n",
    "    * You can represent a summation operator eg: $h = \\sum\\limits_{i = 0}^{m-1} 2i$ in code as follows:\n",
    "     ```python \n",
    "    h = 0\n",
    "    for i in range(m):\n",
    "        h = h + 2*i\n",
    "    ```\n",
    "    \n",
    "    * In this case, you can iterate over all the examples in `x` using a for loop and for each example, keep adding the gradient from that example to the variables `dj_dw` and `dj_db` which are initialized outside the loop. \n",
    "\n",
    "   * Then, you can return `dj_dw` and `dj_db` both divided by `m`.    \n",
    "    <details>\n",
    "          <summary><font size=\"2\" color=\"darkblue\"><b> Click for more hints</b></font></summary>\n",
    "        \n",
    "    * Here's how you can structure the overall implementation for this function\n",
    "    ```python \n",
    "    def compute_gradient(x, y, w, b): \n",
    "        \"\"\"\n",
    "        Computes the gradient for linear regression \n",
    "        Args:\n",
    "          x (ndarray): Shape (m,) Input to the model (Population of cities) \n",
    "          y (ndarray): Shape (m,) Label (Actual profits for the cities)\n",
    "          w, b (scalar): Parameters of the model  \n",
    "        Returns\n",
    "          dj_dw (scalar): The gradient of the cost w.r.t. the parameters w\n",
    "          dj_db (scalar): The gradient of the cost w.r.t. the parameter b     \n",
    "         \"\"\"\n",
    "    \n",
    "        # Number of training examples\n",
    "        m = x.shape[0]\n",
    "    \n",
    "        # You need to return the following variables correctly\n",
    "        dj_dw = 0\n",
    "        dj_db = 0\n",
    "    \n",
    "        ### START CODE HERE ### \n",
    "        # Loop over examples\n",
    "        for i in range(m):  \n",
    "            # Your code here to get prediction f_wb for the ith example\n",
    "            f_wb = \n",
    "            \n",
    "            # Your code here to get the gradient for w from the ith example \n",
    "            dj_dw_i = \n",
    "        \n",
    "            # Your code here to get the gradient for b from the ith example \n",
    "            dj_db_i = \n",
    "     \n",
    "            # Update dj_db : In Python, a += 1  is the same as a = a + 1\n",
    "            dj_db += dj_db_i\n",
    "        \n",
    "            # Update dj_dw\n",
    "            dj_dw += dj_dw_i\n",
    "    \n",
    "        # Divide both dj_dw and dj_db by m\n",
    "        dj_dw = dj_dw / m\n",
    "        dj_db = dj_db / m\n",
    "        ### END CODE HERE ### \n",
    "        \n",
    "        return dj_dw, dj_db\n",
    "    ```\n",
    "    \n",
    "    If you're still stuck, you can check the hints presented below to figure out how to calculate `f_wb` and `cost`.\n",
    "    \n",
    "    <details>\n",
    "          <summary><font size=\"2\" color=\"darkblue\"><b>Hint to calculate f_wb</b></font></summary>\n",
    "           &emsp; &emsp; You did this in the previous exercise! For scalars $a$, $b$ and $c$ (<code>x[i]</code>, <code>w</code> and <code>b</code> are all scalars), you can calculate the equation $h = ab + c$ in code as <code>h = a * b + c</code>\n",
    "          <details>\n",
    "              <summary><font size=\"2\" color=\"blue\"><b>&emsp; &emsp; More hints to calculate f</b></font></summary>\n",
    "               &emsp; &emsp; You can compute f_wb as <code>f_wb = w * x[i] + b </code>\n",
    "           </details>\n",
    "    </details>\n",
    "        \n",
    "    <details>\n",
    "          <summary><font size=\"2\" color=\"darkblue\"><b>Hint to calculate dj_dw_i</b></font></summary>\n",
    "           &emsp; &emsp; For scalars $a$, $b$ and $c$ (<code>f_wb</code>, <code>y[i]</code> and <code>x[i]</code> are all scalars), you can calculate the equation $h = (a - b)c$ in code as <code>h = (a-b)*c</code>\n",
    "          <details>\n",
    "              <summary><font size=\"2\" color=\"blue\"><b>&emsp; &emsp; More hints to calculate f</b></font></summary>\n",
    "               &emsp; &emsp; You can compute dj_dw_i as <code>dj_dw_i = (f_wb - y[i]) * x[i] </code>\n",
    "           </details>\n",
    "    </details>\n",
    "        \n",
    "    <details>\n",
    "          <summary><font size=\"2\" color=\"darkblue\"><b>Hint to calculate dj_db_i</b></font></summary>\n",
    "             &emsp; &emsp; You can compute dj_db_i as <code> dj_db_i = f_wb - y[i] </code>\n",
    "    </details>\n",
    "        \n",
    "    </details>\n",
    "\n",
    "</details>\n",
    "\n",
    "    \n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Run the cells below to check your implementation of the `compute_gradient` function with two different initializations of the parameters $w$,$b$."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compute and display gradient with w initialized to zeroes\n",
    "initial_w = 0\n",
    "initial_b = 0\n",
    "\n",
    "tmp_dj_dw, tmp_dj_db = compute_gradient(x_train, y_train, initial_w, initial_b)\n",
    "print('Gradient at initial w, b (zeros):', tmp_dj_dw, tmp_dj_db)\n",
    "\n",
    "compute_gradient_test(compute_gradient)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now let's run the gradient descent algorithm implemented above on our dataset.\n",
    "\n",
    "**Expected Output**:\n",
    "<table>\n",
    "  <tr>\n",
    "    <td> <b>Gradient at initial , b (zeros)<b></td>\n",
    "    <td> -65.32884975 -5.83913505154639</td> \n",
    "  </tr>\n",
    "</table>"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compute and display cost and gradient with non-zero w\n",
    "test_w = 0.2\n",
    "test_b = 0.2\n",
    "tmp_dj_dw, tmp_dj_db = compute_gradient(x_train, y_train, test_w, test_b)\n",
    "\n",
    "print('Gradient at test w, b:', tmp_dj_dw, tmp_dj_db)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Expected Output**:\n",
    "<table>\n",
    "  <tr>\n",
    "    <td> <b>Gradient at test w<b></td>\n",
    "    <td> -47.41610118 -4.007175051546391</td> \n",
    "  </tr>\n",
    "</table>"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "<a name=\"2.6\"></a>\n",
    "### 2.6 Learning parameters using batch gradient descent \n",
    "\n",
    "You will now find the optimal parameters of a linear regression model by using batch gradient descent. Recall batch refers to running all the examples in one iteration.\n",
    "- You don't need to implement anything for this part. Simply run the cells below. \n",
    "\n",
    "- A good way to verify that gradient descent is working correctly is to look\n",
    "at the value of $J(w,b)$ and check that it is decreasing with each step. \n",
    "\n",
    "- Assuming you have implemented the gradient and computed the cost correctly and you have an appropriate value for the learning rate alpha, $J(w,b)$ should never increase and should converge to a steady value by the end of the algorithm."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def gradient_descent(x, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters): \n",
    "    \"\"\"\n",
    "    Performs batch gradient descent to learn theta. Updates theta by taking \n",
    "    num_iters gradient steps with learning rate alpha\n",
    "    \n",
    "    Args:\n",
    "      x :    (ndarray): Shape (m,)\n",
    "      y :    (ndarray): Shape (m,)\n",
    "      w_in, b_in : (scalar) Initial values of parameters of the model\n",
    "      cost_function: function to compute cost\n",
    "      gradient_function: function to compute the gradient\n",
    "      alpha : (float) Learning rate\n",
    "      num_iters : (int) number of iterations to run gradient descent\n",
    "    Returns\n",
    "      w : (ndarray): Shape (1,) Updated values of parameters of the model after\n",
    "          running gradient descent\n",
    "      b : (scalar)                Updated value of parameter of the model after\n",
    "          running gradient descent\n",
    "    \"\"\"\n",
    "    \n",
    "    # number of training examples\n",
    "    m = len(x)\n",
    "    \n",
    "    # An array to store cost J and w's at each iteration — primarily for graphing later\n",
    "    J_history = []\n",
    "    w_history = []\n",
    "    w = copy.deepcopy(w_in)  #avoid modifying global w within function\n",
    "    b = b_in\n",
    "    \n",
    "    for i in range(num_iters):\n",
    "\n",
    "        # Calculate the gradient and update the parameters\n",
    "        dj_dw, dj_db = gradient_function(x, y, w, b )  \n",
    "\n",
    "        # Update Parameters using w, b, alpha and gradient\n",
    "        w = w - alpha * dj_dw               \n",
    "        b = b - alpha * dj_db               \n",
    "\n",
    "        # Save cost J at each iteration\n",
    "        if i<100000:      # prevent resource exhaustion \n",
    "            cost =  cost_function(x, y, w, b)\n",
    "            J_history.append(cost)\n",
    "\n",
    "        # Print cost every at intervals 10 times or as many iterations if < 10\n",
    "        if i% math.ceil(num_iters/10) == 0:\n",
    "            w_history.append(w)\n",
    "            print(f\"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   \")\n",
    "        \n",
    "    return w, b, J_history, w_history #return w and J,w history for graphing"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now let's run the gradient descent algorithm above to learn the parameters for our dataset."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# initialize fitting parameters. Recall that the shape of w is (n,)\n",
    "initial_w = 0.\n",
    "initial_b = 0.\n",
    "\n",
    "# some gradient descent settings\n",
    "iterations = 1500\n",
    "alpha = 0.01\n",
    "\n",
    "w,b,_,_ = gradient_descent(x_train ,y_train, initial_w, initial_b, \n",
    "                     compute_cost, compute_gradient, alpha, iterations)\n",
    "print(\"w,b found by gradient descent:\", w, b)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Expected Output**:\n",
    "<table>\n",
    "  <tr>\n",
    "    <td> <b> w, b found by gradient descent<b></td>\n",
    "    <td> 1.16636235 -3.63029143940436</td> \n",
    "  </tr>\n",
    "</table>"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We will now use the final parameters from gradient descent to plot the linear fit. \n",
    "\n",
    "Recall that we can get the prediction for a single example $f(x^{(i)})= wx^{(i)}+b$. \n",
    "\n",
    "To calculate the predictions on the entire dataset, we can loop through all the training examples and calculate the prediction for each example. This is shown in the code block below."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "m = x_train.shape[0]\n",
    "predicted = np.zeros(m)\n",
    "\n",
    "for i in range(m):\n",
    "    predicted[i] = w * x_train[i] + b"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We will now plot the predicted values to see the linear fit."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plot the linear fit\n",
    "plt.plot(x_train, predicted, c = \"b\")\n",
    "\n",
    "# Create a scatter plot of the data. \n",
    "plt.scatter(x_train, y_train, marker='x', c='r') \n",
    "\n",
    "# Set the title\n",
    "plt.title(\"Profits vs. Population per city\")\n",
    "# Set the y-axis label\n",
    "plt.ylabel('Profit in $10,000')\n",
    "# Set the x-axis label\n",
    "plt.xlabel('Population of City in 10,000s')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Your final values of $w,b$ can also be used to make predictions on profits. Let's predict what the profit would be in areas of 35,000 and 70,000 people. \n",
    "\n",
    "- The model takes in population of a city in 10,000s as input. \n",
    "\n",
    "- Therefore, 35,000 people can be translated into an input to the model as `np.array([3.5])`\n",
    "\n",
    "- Similarly, 70,000 people can be translated into an input to the model as `np.array([7.])`\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "predict1 = 3.5 * w + b\n",
    "print('For population = 35,000, we predict a profit of $%.2f' % (predict1*10000))\n",
    "\n",
    "predict2 = 7.0 * w + b\n",
    "print('For population = 70,000, we predict a profit of $%.2f' % (predict2*10000))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Expected Output**:\n",
    "<table>\n",
    "  <tr>\n",
    "    <td> <b> For population = 35,000, we predict a profit of<b></td>\n",
    "    <td> $4519.77 </td> \n",
    "  </tr>\n",
    "  \n",
    "  <tr>\n",
    "    <td> <b> For population = 70,000, we predict a profit of<b></td>\n",
    "    <td> $45342.45 </td> \n",
    "  </tr>\n",
    "</table>"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

NameError: name 'null' is not defined